In [2]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
import pickle
from keras.optimizers import Adam
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import time
import random
from keras.utils import np_utils
#import tensorflow.compat.v1 as tf
#tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True))
#sess = print(tf.Session(config=tf.ConfigProto(log_device_placement=True)))


In [6]:
start_time = time.clock()
np.random.seed(7)
random.seed(7)
os.chdir(r'/home/jupyter/Combined Trajectory_Label_Geolife')
filename = '../Combined Trajectory_Label_Geolife/Revised_KerasData_Smoothing.pickle'
with open(filename, mode='rb') as f:
    TotalInput, FinalLabel = pickle.load(f, encoding='latin1')  # Also can use the encoding 'iso-8859-1'

NoClass = len(list(set(np.ndarray.flatten(FinalLabel))))
Threshold = len(TotalInput[0, 0, :, 0])

# Making training and test data: 80% Training, 20% Test
Train_X, Test_X, Train_Y, Test_Y_ori = train_test_split(TotalInput, FinalLabel, test_size=0.20, random_state=7)

Train_Y = np_utils.to_categorical(Train_Y, num_classes=NoClass)
Test_Y = np_utils.to_categorical(Test_Y_ori, num_classes=NoClass)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


In [4]:
# Model and Compile
model = Sequential()

model.add(Conv2D(32, (1, 3), strides=(1, 1), padding='same', activation='relu', input_shape=(1, Threshold, 4)))
model.add(Conv2D(32, (1, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

model.add(Conv2D(64, (1, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(Conv2D(64, (1, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))

model.add(Conv2D(128, (1, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(Conv2D(128, (1, 3), strides=(1, 1), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(1, 2)))
model.add(Dropout(.5))

model.add(Flatten())
A = model.output_shape
model.add(Dense(int(A[1] * 1/4.), activation='relu'))
model.add(Dropout(.5))

model.add(Dense(NoClass, activation='softmax'))

optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Ensemble configuration
score = []
Pred = []
for i in range(7):
    np.random.seed((i + 5) * 2)#1->5
    Number = np.random.choice(len(Train_X), size=len(Train_X), replace=True, p=None)
    Ens_Train_X = np.zeros((len(Train_X), 1, Threshold, 4), dtype=float)
    Ens_Train_Y = np.zeros((len(Train_Y), NoClass), dtype=float)
    counter = 0
    for j in Number:
        Ens_Train_X[counter, :, :, :] = Train_X[j, :, :, :]
        Ens_Train_Y[counter, :] = Train_Y[j, :]
        counter += 1

    #Ens_Train_Y = np_utils.to_categorical(Ens_Train_Y, num_classes=NoClass)
    
    #Ens_Train_X, testX, Ens_Train_Y, testy = train_test_split(Train_X, Train_Y, test_size=0.0001)
    
    model.fit(Ens_Train_X, Ens_Train_Y, epochs=62, batch_size=64, shuffle=False)
    score.append(model.evaluate(Test_X, Test_Y, batch_size=64))
    Pred.append(model.predict(Test_X, batch_size=64))
    # save model
    filename = '../models/model_CNN_' + str(i + 1) + '.h5'
    model.save(filename)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  """Entry point for launching an IPython kernel.


Epoch 1/62
406/406 [==============================] - 23s 53ms/step - loss: 1.1173 - accuracy: 0.5948
Epoch 2/62
406/406 [==============================] - 21s 53ms/step - loss: 0.7760 - accuracy: 0.7398
Epoch 3/62
406/406 [==============================] - 21s 53ms/step - loss: 0.7200 - accuracy: 0.7602
Epoch 4/62
406/406 [==============================] - 21s 53ms/step - loss: 0.6799 - accuracy: 0.7753
Epoch 5/62
406/406 [==============================] - 21s 53ms/step - loss: 0.6352 - accuracy: 0.7918
Epoch 6/62
406/406 [==============================] - 21s 53ms/step - loss: 0.5971 - accuracy: 0.8036
Epoch 7/62
406/406 [==============================] - 21s 53ms/step - loss: 0.5696 - accuracy: 0.8129
Epoch 8/62
406/406 [==============================] - 21s 53ms/step - loss: 0.5374 - accuracy: 0.8251
Epoch 9/62
406/406 [==============================] - 21s 52ms/step - loss: 0.5095 - accuracy: 0.8359
Epoch 10/62
406/406 [==============================] - 21s 53ms/step - loss: 0.479

In [8]:
# Saving the test and training score for varying number of epochs.
#with open('../models/ensemble_Motion_2_.pickle', 'wb') as f:
#    pickle.dump([score,Pred], f)

In [18]:
with open('../models/ensemble_GIS_69.pickle', 'rb') as f:
    score,Pred = pickle.load(f, encoding='latin1')

In [19]:
score

[[1.0786223411560059, 0.5676633715629578],
 [1.0514806509017944, 0.6128236651420593],
 [1.0839840173721313, 0.6297780275344849],
 [1.1392892599105835, 0.6676942110061646],
 [1.0483860969543457, 0.6228421926498413],
 [1.1042126417160034, 0.6727805137634277],
 [1.2476413249969482, 0.6786375045776367]]

In [20]:
# Calculating the accuracy, precision, recall.
CombinedPred = np.mean(Pred, axis=0)
Pred_Label = np.argmax(CombinedPred, axis=1)

counter = 0
for i in range(len(Pred_Label)):
    if Pred_Label[i] == Test_Y_ori[i]:
        counter += 1
EnsembleAccuracy = counter * 1./len(Pred_Label)

PredictedPositive = []
for i in range(NoClass):
    AA = np.where(Pred_Label == i)[0]
    PredictedPositive.append(AA)

ActualPositive = []
for i in range(NoClass):
    AA = np.where(Test_Y_ori == i)[0]
    ActualPositive.append(AA)

TruePositive = []
FalsePositive = []
for i in range(NoClass):
    AA = []
    BB = []
    for j in PredictedPositive[i]:
        if Pred_Label[j] == Test_Y_ori[j]:
            AA.append(j)
        else:
            BB.append(j)
    TruePositive.append(AA)
    FalsePositive.append(BB)
Precision = []
Recall = []
for i in range(NoClass):
    Precision.append(len(TruePositive[i]) * 1./len(PredictedPositive[i]))
    Recall.append(len(TruePositive[i]) * 1./len(ActualPositive[i]))

ConfusionM = confusion_matrix(list(Test_Y_ori), Pred_Label, labels=range(NoClass))

print(score)
print('Ensemble Accuracy: ', EnsembleAccuracy)
print('Confusion Matrix: ', ConfusionM)
print("Recall", Recall)
print('Precision', Precision)
print(time.clock() - start_time, "seconds")

[[1.0786223411560059, 0.5676633715629578], [1.0514806509017944, 0.6128236651420593], [1.0839840173721313, 0.6297780275344849], [1.1392892599105835, 0.6676942110061646], [1.0483860969543457, 0.6228421926498413], [1.1042126417160034, 0.6727805137634277], [1.2476413249969482, 0.6786375045776367]]
Ensemble Accuracy:  0.6915844636251541
Confusion Matrix:  [[1895   92   77   10   48]
 [ 414  609   64   30    4]
 [ 321   67  950   65   36]
 [ 147   48  267  304  122]
 [ 141   16   16   16  729]]
Recall [0.8930254476908577, 0.5432649420160571, 0.660180681028492, 0.34234234234234234, 0.7941176470588235]
Precision [0.6494174091843729, 0.7319711538461539, 0.6914119359534207, 0.7152941176470589, 0.7763578274760383]
0.6080999999999994 seconds


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:46: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


In [21]:
print(sum(Recall)/len(Recall))
print(sum(Precision)/len(Precision))

0.6465862120273146
0.7128904888214089


In [23]:
sum(F_score)/len(F_score)

0.6598549056279109